In [590]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import time
import pandas as pd
import os
import jieba
path = "./input_data.csv"

In [591]:
# LeNET
# class CNN(nn.Module):
#     def __init__(self, word_dim, n_class, num_words): # 词向量维, 类别, 词个数
#         super(CNN, self).__init__()
#         self.embed = nn.Sequential(
#             nn.Embedding(num_words, word_dim),# 100维词向量
#         )
#         self.conv = nn.Sequential(
#             nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(3, 3), stride=1),#  15*100
#             nn.ReLU(True),
#             nn.MaxPool2d(kernel_size=2, stride=2), #7*49
#         )
        
#         self.Fnn = nn.Sequential(
#             nn.Dropout(),
#             nn.Linear(2664, 400),
#             nn.ReLU(True),
#             nn.Dropout(),
#             nn.Linear(400, 84),
#             nn.ReLU(True),
#             nn.Linear(84, n_class),
#         )
#     def forward(self, x):
#         x = self.embed(x)
#         x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2] ))
#         x = self.conv(x)
#         x = x.view(x.size(0), -1)
#         return self.Fnn(x)
class RNN(nn.Module):
    def __init__(self, num_words, word_dim, hiden_dim, n_layer, n_class):
        super(RNN, self).__init__()
        self.embed = nn.Sequential(
            nn.Embedding(num_words, word_dim),# 100维词向量
        )
#         self.lstm = nn.LSTM(word_dim, hiden_dim, n_layer)
        self.lstm = nn.GRU(word_dim, hiden_dim, n_layer) 
        self.classifier = nn.Linear(hiden_dim, n_class)
    def forward(self, x):
        x = self.embed(x)
        out, _ = self.lstm(x)
        out = out[:,-1,:]
        return self.classifier(out)

### 加载数据，对文本预处理

In [592]:
Data = pd.read_csv(path).values
Origin_inputX, Origin_inputY = Data[:, 1], Data[:, 2]
Origin_inputX = [x.split(",") for x in Origin_inputX]
X = set()
Y = set()
for x in Origin_inputX:
    for xx in x:
        X.add(xx)
for y in Origin_inputY:
    Y.add(y)
X_str_num = {key:i for i, key in enumerate(X)}
X_num_str = {X_str_num[key]:key for key in X_str_num}

Y_str_num = {key:i for i, key in enumerate(Y)}
Y_num_str = {Y_str_num[key]:key for key in Y_str_num}
Origin_inputY = [Y_str_num[y] for y in Origin_inputY]

# dicts = []
# embed = nn.Embedding()

In [593]:
x = torch.Tensor(torch.rand(2,3))
print(x)
x = x.reshape((x.shape[0], 1, x.shape[1]))
x

tensor([[0.8420, 0.5732, 0.4973],
        [0.7696, 0.4802, 0.5091]])


tensor([[[0.8420, 0.5732, 0.4973]],

        [[0.7696, 0.4802, 0.5091]]])

In [594]:
Max_len = max([len(i) for i in Origin_inputX])
for i in range(len(Origin_inputX)):
    for j in range(len(Origin_inputX[i])):
        if Origin_inputX[i][j] in X_str_num:
            Origin_inputX[i][j] = X_str_num[Origin_inputX[i][j]]
        else:
            print("{} not in keys".format(Origin_inputX[i][j]))
    while len(Origin_inputX[i]) < Max_len:
        Origin_inputX[i].append(0)
Origin_inputX, Origin_inputY

([[296, 317, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [554, 318, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [50, 699, 546, 83, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [887, 867, 888, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [191, 1014, 837, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [887, 77, 1025, 595, 888, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [700, 1007, 549, 119, 1068, 191, 156, 0, 0, 0, 0, 0, 0, 0, 0],
  [574, 695, 76, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [686, 418, 948, 837, 574, 695, 76, 0, 0, 0, 0, 0, 0, 0, 0],
  [527, 958, 994, 240, 148, 422, 837, 574, 695, 76, 0, 0, 0, 0, 0],
  [527, 148, 422, 837, 574, 695, 76, 0, 0, 0, 0, 0, 0, 0, 0],
  [1011, 150, 148, 422, 837, 574, 695, 76, 0, 0, 0, 0, 0, 0, 0],
  [527, 958, 994, 240, 150, 148, 422, 837, 574, 695, 76, 0, 0, 0, 0],
  [606, 148, 422, 837, 574, 695, 76, 0, 0, 0, 0, 0, 0, 0, 0],
  [1011, 150, 422, 837, 574, 695, 76, 0, 0, 0, 0, 0, 0, 0, 0],
  [527, 958, 994, 240, 150, 422, 837, 574, 695, 76, 0, 0, 0, 0, 0],
  [606, 422, 837, 574, 695, 76

In [595]:
Origin_inputX = [torch.LongTensor(x) for x in Origin_inputX]
Origin_inputY = [torch.LongTensor([x]) for x in Origin_inputY]
Origin_inputX, Origin_inputY

([tensor([296, 317,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0]),
  tensor([554, 318,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0]),
  tensor([ 50, 699, 546,  83,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0]),
  tensor([887, 867, 888,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0]),
  tensor([ 191, 1014,  837,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0]),
  tensor([ 887,   77, 1025,  595,  888,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0]),
  tensor([ 700, 1007,  549,  119, 1068,  191,  156,    0,    0,    0,    0,    0,
             0,    0,    0]),
  tensor([574, 695,  76,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0]),
  tensor([686, 418, 948, 837, 574, 695,  76,   0,   0,   0,   0,   0,   0,   0,
            0]),
  tensor([527, 958, 994, 240, 148, 422, 837, 574, 695,  76,   0,   0,   0,   0,
  

In [596]:
embedding = nn.Embedding(10, 3)
input = Variable(torch.LongTensor([[1,2,4,5],[4,3,2,1]]))
embedding(input)


tensor([[[-0.9561, -0.3784, -2.6955],
         [ 0.5260, -0.8589, -0.4450],
         [-2.5092, -0.5417,  1.0637],
         [-1.7016,  0.7757,  0.2172]],

        [[-2.5092, -0.5417,  1.0637],
         [ 0.0911,  0.5030, -1.0188],
         [ 0.5260, -0.8589, -0.4450],
         [-0.9561, -0.3784, -2.6955]]], grad_fn=<EmbeddingBackward>)

In [597]:
class Datasets(Dataset):
    def __init__(self, X, Y, transforms=None):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
def rev(batch):
    x, y = zip(*batch)
    
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Origin_inputX, Origin_inputY, test_size=0.1)

In [598]:
# model = CNN(150, len(Y), len(X))#.cuda()
model = RNN(len(X), 150, 200, 2, len(Y))
batch_size = 10
learning_rate = 1e-2
data_tf = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5], [0.5, 0.5])
])
train_data = Datasets(x_train, y_train)
test_data = Datasets(x_test, y_test)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
train_data[0],  train_loader.__iter__().__next__()
# for i, x in enumerate(train_loader):
#     if i == 0:
#         print(i, "--->", x)
#         break

((tensor([ 164,  202, 1105,  628,  837,  826,    0,    0,    0,    0,    0,    0,
             0,    0,    0]), tensor([15])),
 [tensor([[ 539,  695,   76,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [ 887,  911,  903, 1082,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [ 412,  960,  149,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [1170,  695,  735,  191, 1087,  694,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [   7,  422,  837,  415,  471,  695, 1024,    0,    0,    0,    0,    0,
              0,    0,    0],
          [ 882,  268,  790,  837,  605,  546,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [ 695, 1024,    7,  422,  837, 1020,  471,    0,    0,    0,    0,    0,
              0,    0,    0],
          [ 695, 1024,   56,  148,  422,  837,  210,  471,    0,    0,    0,    0,

In [599]:
# if torch.cuda.is_available():
criterion = nn.CrossEntropyLoss()#.cuda()
# else:
#     criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [600]:
num_epoch = 100
for epoch in range(num_epoch):
    print('*' * 10)
    print('epoch {}'.format(epoch + 1))
    since = time.time()
    running_loss, running_acc = 0.0, 0.0
    
    for i, data in enumerate(train_loader, 1):
        # 获取数据
        img, label = data
#         print(img)
#         if torch.cuda.is_available():
#             img, label = Variable(img).cuda(), Variable(label).cuda()
#         else:
        img, label = Variable(img), Variable(label)
        # 前向传播
        out = model(img)
        label = label.reshape(label.shape[0])
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().item()
        running_acc += num_correct
        # 后向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 一共6w个数据，每5k 输出一批数据上的精度
        if i % 20 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoch, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    # 每批训练输出整个数据集上的精度
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_data)), running_acc / (len(
            train_data))))
    # 测试
    model.eval()
    eval_loss, eval_acc = 0.0, 0.0
    for data in test_loader:
        img, label = data
#         img = img.view(img.size(0), -1)
        
#         if torch.cuda.is_available():
#             img, label = Variable(img).cuda(), Variable(label).cuda()
#         else:
        img, label = Variable(img), Variable(label)
        
        out = model(img)
        label = label.reshape(label.shape[0])
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().item()
        eval_acc += num_correct
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_data)), eval_acc / (len(test_data))))
    print('Time:{:.1f} s'.format(time.time() - since))
    print()

**********
epoch 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1/100] Loss: 2.697083, Acc: 0.290000
[1/100] Loss: 2.622776, Acc: 0.297500
[1/100] Loss: 2.517607, Acc: 0.321667
[1/100] Loss: 2.493613, Acc: 0.322500
[1/100] Loss: 2.464611, Acc: 0.329000
[1/100] Loss: 2.444075, Acc: 0.330000
[1/100] Loss: 2.428002, Acc: 0.325000
[1/100] Loss: 2.420418, Acc: 0.329375
[1/100] Loss: 2.425979, Acc: 0.321111
[1/100] Loss: 2.417789, Acc: 0.323500
Finish 1 epoch, Loss: 2.414028, Acc: 0.326996


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test Loss: 2.522576, Acc: 0.324786
Time:12.4 s

**********
epoch 2
[2/100] Loss: 2.273034, Acc: 0.310000
[2/100] Loss: 2.288317, Acc: 0.330000
[2/100] Loss: 2.336946, Acc: 0.328333
[2/100] Loss: 2.301804, Acc: 0.347500
[2/100] Loss: 2.277240, Acc: 0.344000
[2/100] Loss: 2.297332, Acc: 0.350833
[2/100] Loss: 2.294807, Acc: 0.353571
[2/100] Loss: 2.326953, Acc: 0.341875
[2/100] Loss: 2.325048, Acc: 0.342778
[2/100] Loss: 2.325121, Acc: 0.341000
Finish 2 epoch, Loss: 2.331897, Acc: 0.339354
Test Loss: 2.423707, Acc: 0.239316
Time:10.8 s

**********
epoch 3
[3/100] Loss: 2.527431, Acc: 0.290000
[3/100] Loss: 2.346062, Acc: 0.322500
[3/100] Loss: 2.371998, Acc: 0.328333
[3/100] Loss: 2.334723, Acc: 0.323750
[3/100] Loss: 2.301914, Acc: 0.327000
[3/100] Loss: 2.306385, Acc: 0.322500
[3/100] Loss: 2.280641, Acc: 0.330714
[3/100] Loss: 2.305773, Acc: 0.332500
[3/100] Loss: 2.324804, Acc: 0.328333
[3/100] Loss: 2.322222, Acc: 0.329000
Finish 3 epoch, Loss: 2.321676, Acc: 0.331274
Test Loss: 2.5

[34/100] Loss: 2.280009, Acc: 0.347500
[34/100] Loss: 2.262828, Acc: 0.350714
[34/100] Loss: 2.294360, Acc: 0.349375
[34/100] Loss: 2.283567, Acc: 0.350000
[34/100] Loss: 2.291883, Acc: 0.348000
Finish 34 epoch, Loss: 2.284110, Acc: 0.346483
Test Loss: 2.609506, Acc: 0.324786
Time:13.2 s

**********
epoch 35
[35/100] Loss: 2.528049, Acc: 0.335000
[35/100] Loss: 2.427476, Acc: 0.332500
[35/100] Loss: 2.413604, Acc: 0.313333
[35/100] Loss: 2.401866, Acc: 0.308750
[35/100] Loss: 2.340481, Acc: 0.329000
[35/100] Loss: 2.337158, Acc: 0.322500
[35/100] Loss: 2.356213, Acc: 0.321429
[35/100] Loss: 2.330552, Acc: 0.325625
[35/100] Loss: 2.322627, Acc: 0.331111
[35/100] Loss: 2.306162, Acc: 0.332500
Finish 35 epoch, Loss: 2.292450, Acc: 0.332700
Test Loss: 2.595686, Acc: 0.324786
Time:13.9 s

**********
epoch 36
[36/100] Loss: 2.229258, Acc: 0.355000
[36/100] Loss: 2.283619, Acc: 0.355000
[36/100] Loss: 2.312741, Acc: 0.346667
[36/100] Loss: 2.287059, Acc: 0.357500
[36/100] Loss: 2.303637, Acc:

[66/100] Loss: 2.266213, Acc: 0.356000
Finish 66 epoch, Loss: 2.265091, Acc: 0.357414
Test Loss: 2.523296, Acc: 0.324786
Time:16.1 s

**********
epoch 67
[67/100] Loss: 2.248124, Acc: 0.400000
[67/100] Loss: 2.170154, Acc: 0.377500
[67/100] Loss: 2.167681, Acc: 0.365000
[67/100] Loss: 2.201141, Acc: 0.365000
[67/100] Loss: 2.232688, Acc: 0.355000
[67/100] Loss: 2.280230, Acc: 0.348333
[67/100] Loss: 2.266389, Acc: 0.351429
[67/100] Loss: 2.263001, Acc: 0.353750
[67/100] Loss: 2.282622, Acc: 0.352778
[67/100] Loss: 2.271609, Acc: 0.350000
Finish 67 epoch, Loss: 2.265331, Acc: 0.352662
Test Loss: 2.609317, Acc: 0.324786
Time:16.4 s

**********
epoch 68
[68/100] Loss: 2.129305, Acc: 0.410000
[68/100] Loss: 2.217218, Acc: 0.362500
[68/100] Loss: 2.205978, Acc: 0.383333
[68/100] Loss: 2.270910, Acc: 0.358750
[68/100] Loss: 2.266597, Acc: 0.354000
[68/100] Loss: 2.273488, Acc: 0.341667
[68/100] Loss: 2.286696, Acc: 0.346429
[68/100] Loss: 2.289144, Acc: 0.330625
[68/100] Loss: 2.265803, Acc:

[99/100] Loss: 2.259028, Acc: 0.285000
[99/100] Loss: 2.224208, Acc: 0.315000
[99/100] Loss: 2.240900, Acc: 0.317500
[99/100] Loss: 2.251441, Acc: 0.332000
[99/100] Loss: 2.240105, Acc: 0.340000
[99/100] Loss: 2.228930, Acc: 0.351429
[99/100] Loss: 2.226484, Acc: 0.350625
[99/100] Loss: 2.248512, Acc: 0.352778
[99/100] Loss: 2.271196, Acc: 0.345500
Finish 99 epoch, Loss: 2.279594, Acc: 0.346483
Test Loss: 2.627505, Acc: 0.324786
Time:16.7 s

**********
epoch 100
[100/100] Loss: 2.308085, Acc: 0.390000
[100/100] Loss: 2.301314, Acc: 0.382500
[100/100] Loss: 2.272845, Acc: 0.391667
[100/100] Loss: 2.278228, Acc: 0.370000
[100/100] Loss: 2.274791, Acc: 0.365000
[100/100] Loss: 2.277286, Acc: 0.360000
[100/100] Loss: 2.273535, Acc: 0.357857
[100/100] Loss: 2.269443, Acc: 0.356250
[100/100] Loss: 2.279623, Acc: 0.347222
[100/100] Loss: 2.292237, Acc: 0.344500
Finish 100 epoch, Loss: 2.282890, Acc: 0.346008
Test Loss: 2.622065, Acc: 0.324786
Time:17.6 s

